
<img src="http://hydrology.usu.edu/images/taudem5logo.gif" style="float:left;width:300px;padding:20px">       


# TauDEM Raster Processing Notebook

This notebook demonstrates basic watershed processing on using the TauDEM library.  For more information about TauDEM, see http://hydrology.usu.edu/taudem/taudem5/


## Script Setup

In [ ]:
# import required libaries for geoprocessing
%matplotlib inline
import os
import shutil
import hs_utils
import taudem

# create an instance of the hydroshare rest interface
hs = hs_utils.hydroshare()

### Retrieve a raster resource using its ID

The raster resouce id can be found by navigating to a resource in HydroShare and selecting the ID from the URL.

In [ ]:
# get some resource content. The resource content is returned as a dictionary
content = hs.getResourceFromHydroShare(os.environ['HS_RES_ID'])


### Prepare a directory for output files

In [ ]:
# prepare a clean workspace for TauDEM output files
data_directory = taudem.create_workspace('logan')   

---


##  Raster Processing

The following operations demonstrate how TauDEM can be used to perform basic raster processes. 
http://hydrology.usu.edu/taudem/taudem5/TauDEM53CommandLineGuide.pdf

### 1. View the Raw DEM

In [ ]:
# display the raw dem
raw_dem_path = hs.content['logan.tif']
taudem.plot_tiff(raw_dem_path)

### 2. Remove DEM Pits

In [ ]:
# Fill the DEM Pits

# set the output paths
fill = os.path.join(data_directory, 'loganfel.tif')
cmd = 'pitremove -z %s -fel %s' % (raw_dem_path, fill)
taudem.run_cmd(cmd, processors=4)

### 3. Calculate Flow Directions

In [ ]:
#####################
# D8 Flow Direction #
#####################

# set the output paths 
fdr = os.path.join(data_directory, 'fdr.tif')  # flowdir
sd8 = os.path.join(data_directory, 'sd8.tif')  # slope

cmd = 'd8flowdir -fel %s -sd8 %s -p %s' % (fill, sd8, fdr)
taudem.run_cmd(cmd, processors=4)


In [ ]:
#######################
# DInf Flow direction #
#######################

# set the output paths 
ang = os.path.join(data_directory, 'loganang.tif')  # flow angle
slp = os.path.join(data_directory, 'loganslp.tif')  # flow slope

cmd = 'dinfflowdir -fel %s -ang %s -slp %s' % (fill, ang, slp)
taudem.run_cmd(cmd, processors=4)


#### Plot the Results

In [ ]:
taudem.plot_tiff(tiff=[fdr, ang], title=['D8 Flow Direction', 'DInf Flow Angle'])
taudem.plot_tiff(tiff=[sd8, slp], title=['D8 Slope', 'DInf Slope'])

### 4. Calculate Contributing Area

In [ ]:
########################
# D8 Contributing Area #
########################

# set the output paths
ad8 = os.path.join(data_directory, 'loganad8.tif')  # D8 contributing area

# build the taudem command
cmd = 'aread8 -p %s -ad8 %s' % (fdr, ad8)
# -p:   D8 flow direction output file   
# -ad8: D8 contributing area file (output)  

# run the command
taudem.run_cmd(cmd, processors=4)

In [ ]:
##########################
# DInf Contributing Area #
##########################

# set the output paths
sca = os.path.join(data_directory, 'logansca.tif')  # D-Infinity contributing area

# build the taudem command
cmd = 'areadinf -ang %s -sca %s' % (ang, sca)                         
# -ang: D-infinity flow direction input file.                                  
# -sca: D-infinity area output file.                                             

# run the command
taudem.run_cmd(cmd, processors=4)

Download another resource from HydroShare that contains the outlet location of the Logan watershed.  This resource is located at the following address [https://www.hydroshare.org/resource/1461c298e8594eb48d28e317f400c728/](https://www.hydroshare.org/resource/1461c298e8594eb48d28e317f400c728/)

In [ ]:
# connect with HydroShare and download the Logan Outlet resource
content = hs.getResourceFromHydroShare('0f7489e2ab8e4ad18f7ae1d46e66163d')

# unzip the content for this resource
import zipfile
zip_ref = zipfile.ZipFile(hs.content['LoganOutlet.zip'], 'r')
zip_ref.extractall(data_directory)
for f in zip_ref.infolist():
    # add the path to the hs.content variable
    filepath = os.path.join(data_directory, f.filename)
    hs.content[f.filename] = filepath
zip_ref.close()

# print the content files as a dictionary
import hs_utils
hs_utils.display_resource_content_files(hs.content, text='Below are the files that have been loaded into memory:')


In [ ]:
###################################################################
# D8 Contributing Area - Upslope of outlets defined by outlet.shp #
###################################################################

# set the output paths
ad8o = os.path.join(data_directory, 'loganad8o.tif')  # Contributing area upslope from outlet
outlet = hs.content['logan_Outletmv.shp']  # outlet shapefile from hydroshare resource

# build the taudem command
cmd = 'aread8 -p %s -o %s -ad8 %s' % (fdr, outlet, ad8o)
# -p:   D8 flow direction output file 
# -o:   outlet shapefile.   
# -ad8: D8 contributing area file (output)  

# run the command
taudem.run_cmd(cmd, processors=4)

#### Plot the Results

In [ ]:
# plot the contributing area result
taudem.plot_tiff(tiff=[ad8, sca, ad8o], 
                 title=['D8 Contributing Area', 
                        'DInf Contributing Area',
                        'D8 Contributing Area from Outlet'])

### 5. Calculate Grid Network

In [ ]:
###############
# Grid Netork #
###############

# set the output paths
plen = os.path.join(data_directory, 'loganplen.tif')  # Longest flow path
tlen = os.path.join(data_directory, 'logantlen.tif')  # Total flow path length
gord = os.path.join(data_directory, 'logangord.tif')  # Grid network order

# build the taudem command
cmd = 'gridnet -p %s -plen %s -tlen %s -gord %s' % (fdr, plen, tlen, gord)                           
# -p:     D8 flow direction input file.                                            
# -plen:  the longest flow length upstream of each point (output)
# -tlen:  the total path length upstream of each point (output)                 
# -gord:  the grid of strahler order (output)                            
            
# run the command
taudem.run_cmd(cmd, processors=4)


#### Plot the Results

In [ ]:
# plot the results
taudem.plot_tiff(tiff=[plen, tlen, gord],
                title=['Longest Flow Path',
                       'Total Flow Path Length',
                       'Grid Network'])



### 6. Calculate Peuker Douglas 

In [ ]:
##################
# Peuker Douglas #
##################

# set the output paths
ss = os.path.join(data_directory, 'loganss.tif')  # Stream network skeleton

# build the taudem command
cmd = 'peukerdouglas -fel %s -ss %s' % (fill, ss) 

# run the command
taudem.run_cmd(cmd, processors=4)


#### Plot the Results

In [ ]:
# plot the results
taudem.plot_tiff(tiff=ss, title='Stream Network Skeleton')

### 7. PeukerDouglas stream delineation

In [ ]:
# set the output paths
ssa = os.path.join(data_directory, 'loganssa.tif')  
drp = os.path.join(data_directory, 'logandrp.txt')  
src = os.path.join(data_directory, 'logansrc.tif') 

In [ ]:
########################
# Contributing Area D8 #
########################

# build the taudem command
cmd = 'aread8 -p %s -o %s -ad8 %s -wg %s' % (fdr, outlet, ssa ,ss) 
# -p:   D8 flow direction output file 
# -o:   the optional outlet shape input file.   
# -ad8: D8 contributing area file (output)  
# -wg:  is the optional weight grid input file. 

# run the command
taudem.run_cmd(cmd, processors=4)

In [ ]:
########################
# Stream Drop Analysis #
########################

# build the taudem command
cmd = 'dropanalysis -p %s -fel %s -ad8 %s -ssa %s -drp %s -o %s -par 5 500 10 0' \
       % (fdr, fill, ad8, ssa, drp, outlet)        
# -ad8: input contributing area file used in calculations of drainage density. 
# -p:   input D8 flow directions file.  
# -fel: input elevation file.   
# -ssa: accumulated stream source file 
# -drp: a text file for drop analysis tabular output 
# -o:   the shapefile containing input outlets  
# -par: <lower bound> <upper bound> <number of threshholds> <steptype> (0=log, 1=arithmetic)

# run the command
taudem.run_cmd(cmd, processors=4)

In [ ]:
####################
# Drop Thesholding #
####################

# build the taudem command
cmd = 'threshold -ssa %s -src %s -thresh 300' % (ssa, src)  
# -ssa:    the file to be thresholded                                             
# -src:    the name of file with the thresholded output     
# -thresh: the value of the threshold  

# run the command
taudem.run_cmd(cmd, processors=4)

#### View the Results

In [ ]:
# view the results
with open(drp, 'r') as f:
    text = f.readlines()
    for line in text:
        print(line)

taudem.plot_tiff(tiff=[src], title=['Threshold Streams'])

---
### 8. Stream Network 

In [ ]:
# set the output paths
order = os.path.join(data_directory, 'loganord3.tif')  
tree = os.path.join(data_directory, 'logantree.dat')  
coord = os.path.join(data_directory, 'logancoord.dat') 
net = os.path.join(data_directory, 'logannet.shp') 
watershed = os.path.join(data_directory, 'loganw.tif') 

In [ ]:
##################
# Stream Network #
##################

# build the taudem command 
cmd = 'streamnet -fel %s -p %s -ad8 %s -src %s -ord %s -tree %s -coord %s -net %s -w %s -o %s' \
       % (fill, fdr, ad8, src, order, tree, coord, net, watershed, outlet)
# -fel: pit filled elevation
# -p:  D8 flow direction grid
# -ad8: D8 drainage area
# -src: stream raster grid
# -o: watershed outlet shapefile
# -ord: stream order grid (output)
# -tree: network connectivity tree (output)
# -coord: network coordinates (output)
# -net: stream reach shapefile (output)
# -w: watershed grid (output)

# run the command
taudem.run_cmd(cmd, processors=4)

#### View the Results

In [ ]:
# plot the results
taudem.plot_tiff(watershed)

---
## Save the results back into HydroShare

Using the HydroShare rest api, you can create a new resource in HydroShare from a content file.

In [ ]:
# save this file as a new resource
# lets save this content as a new resource in HydroShare
title = 'DEM derived Logan data'
abstract = 'This a group of files that were derived from the Logan River watershed.'
keywords = ['Demo', 'JupyterHub', 'TauDEM', 'Logan', 'Geoprocessing']
rtype = 'genericresource'
files = [hs.content['logan.tif'],    # input DEM for Logan Watershed
         hs.content['taudem_logan.ipynb'], # this notebook
         fill, # pit fill dem
         fdr,  # D8 flow direction
         sd8,  # D8 slope?
         ad8,  # D8 contributing area
         watershed, # watershed grid
         order, # stream order grid
         hs.content['LoganOutlet.zip']]  

# create a hydroshare resource containing these data
resource_id = hs.createHydroShareResource(abstract, 
                                          title, 
                                          derivedFromId=os.environ['HS_RES_ID'],
                                          keywords=keywords, 
                                          resource_type=rtype, 
                                          content_files=files, 
                                          public=False)